# Test data loading

In [1]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Bokeh Demo

In [2]:
from bokeh.palettes import HighContrast3
from bokeh.plotting import figure, show

fruits = ["Apples", "Pears", "Nectarines", "Plums", "Grapes", "Strawberries"]
years = ["2015", "2016", "2017"]

data = {"fruits": fruits, "2015": [2, 1, 4, 3, 2, 4], "2016": [5, 3, 4, 2, 4, 6], "2017": [3, 2, 4, 4, 5, 3]}

p = figure(x_range=fruits, height=250, title="Fruit Counts by Year", toolbar_location=None, tools="hover", tooltips="$name @fruits: @$name")

p.vbar_stack(years, x="fruits", width=0.9, color=HighContrast3, source=data, legend_label=years)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

In [11]:
import json

with open('data/cleaned_business.json', 'r', encoding='utf-8') as file:
    businesses = json.load(file)

print(businesses[0])
print(businesses[0]['name'])

postal_codes = []
for business in businesses:
    postal_codes.append(business['postal_code'])

print(postal_codes[0:10])

{'business_id': 'MTSW4McQd7CbVtyjqoe9mw', 'name': 'St Honore Pastries', 'address': '935 Race St', 'city': 'Philadelphia', 'state': 'PA', 'postal_code': '19107', 'latitude': 39.9555052, 'longitude': -75.1555641, 'stars': 4.0, 'review_count': 80, 'is_open': 1, 'attributes': {'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': "u'free'", 'Alcohol': "u'none'", 'Caters': 'True'}, 'categories': 'Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries', 'hours': {'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}}
St Honore Pastries
['19107', '19106', '19147', '19147', '19127', '19123', '19146', '19104', '19152', '19

In [12]:
from collections import Counter
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6


postal_codes.sort()
postal_code_counts = Counter(postal_codes)
codes = list(postal_code_counts.keys())
counts = list(postal_code_counts.values())

p = figure(x_range=codes, height=400, title="Distribution of Postal Codes", toolbar_location=None,tools="")

p.vbar(x=codes, top=counts, width=0.9, color="#CAB2D6")
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.axis_label = "Postal Codes"
p.yaxis.axis_label = "Frequency"
p.xaxis.major_label_orientation = 1.3 


show(p)